In [6]:
# BRCF.4.ImputeImplicit.ipynb
# Create Good.Ratings.csv which replace zero ratings with average ratings if available.

In [1]:
import numpy as np
import pandas as pd

In [2]:
# Need to again delete MC.Ratings.csv line #287,498 "178195,1/9565,000229,10"
# Header line from BX-Book-Ratings.csv has been added back to MC.Ratings.csv with comma as separator
df = pd.read_csv("data/MC.Ratings.csv")

In [3]:
# Further clean ISBN
df.ISBN = df.ISBN.str.replace(r'[^\w\d]+', '')

In [4]:
# Compute average rating for each ISBN from available non-zero ratings.
avg_ratings = df.groupby('ISBN')['Book-Rating'].mean().round().astype(np.int8)

In [5]:
len(avg_ratings)

185811

In [6]:
def useAvgRatings(path="") :
	countVal = 0
	countKey = 0
	count1 = 0
	count2 = 0
	count3 = 0
	count4 = 0    
	count5 = 0  
	fratings = open(path+"Good.Ratings.csv", 'w')
	for line in open(path+"BX-Book-Ratings.csv"):
		count1+=1        
		line = line.replace('"', "")
		line = line.replace("\\","")
		line = line.replace("\n","")        
		(user,bookid,rating) = line.split(";")
		try:
			if float(rating) > 0.0: # has user rating
				count2+=1 
			else :
				if bookid in avg_ratings :
					rating = avg_ratings[bookid].astype('str')
					count3+=1                    
				else :
					rating = '0'
					count4+=1                                        
			fratings.write(user + ',' + bookid + ',' + rating + '\n')
			count5+=1            
		except ValueError:
			countVal+=1
#			print "value error found! " + user + bookid + rating
		except KeyError:
			countKey +=1
#			print "key error found! " + user + " " + bookid
	print countVal
	print countKey
	print count1  
	print count2 
	print count3 
	print count4 
	print count5 
	fratings.close()    
	
	return 0

In [7]:
useAvgRatings('data/')

1
0
1149781
433671
494024
222085
1149780


0

In [ ]:
# We read 1,149,781 from BX-Book-Ratings.csv.
# 433,671 have non-zero ratings; thus, no impute needed.
# We use average ratings for 494,024 records.
# We can only use zero rating for 222,085 records as buyers of those books not provide rating.
# We wrote 1,149,780 records to Good.ratings.csv
#
# Thus, we double ratings available for building CF.